In [3]:
import os
import argparse

import cv2

import rosbag
from sensor_msgs.msg import Image
from cv_bridge import CvBridge
    
def extract_images(bag_file,image_topic,ouput_dir):
    bag = rosbag.Bag(bag_file, "r")
    bridge = CvBridge()
    count = 0
    for topic, msg, t in bag.read_messages(topics=[image_topic]):
        cv_img = bridge.imgmsg_to_cv2(msg, desired_encoding="passthrough")

        cv_img = cv2.cvtColor(cv_img, cv2.COLOR_RGB2BGR)
        cv_img = cv2.rotate(cv_img, cv2.ROTATE_180)
        cv2.imwrite(os.path.join(output_dir, "frame_%06i.png" % count), cv_img)
        print ("Wrote image %i" % count)

        count += 1

    bag.close()

In [12]:
from pathlib import Path


bag_name = "2021-12-01-13-00-45"

bag = f"../husky_bags/{bag_name}.bag"

!rosbag info {bag}

topic = "/husky/wrist_realsense/color/image_raw"
output_dir = f"./dataset/{bag_name}/"

Path(output_dir).mkdir(parents=True, exist_ok=True)


extract_images(bag,topic,output_dir)

path:        ../husky_bags/2021-12-01-13-00-45.bag
version:     2.0
duration:    1:59s (119s)
start:       Dec 01 2021 13:00:46.32 (1638360046.32)
end:         Dec 01 2021 13:02:45.94 (1638360165.94)
size:        9.2 GB
messages:    3572
compression: none [3572/3572 chunks]
types:       sensor_msgs/Image [060021388200f6f0f447d0fcd9c64743]
topics:      /husky/wrist_realsense/color/image_raw   3572 msgs    : sensor_msgs/Image
Wrote image 0
Wrote image 1
Wrote image 2
Wrote image 3
Wrote image 4
Wrote image 5
Wrote image 6
Wrote image 7
Wrote image 8
Wrote image 9
Wrote image 10
Wrote image 11
Wrote image 12
Wrote image 13
Wrote image 14
Wrote image 15
Wrote image 16
Wrote image 17
Wrote image 18
Wrote image 19
Wrote image 20
Wrote image 21
Wrote image 22
Wrote image 23
Wrote image 24
Wrote image 25
Wrote image 26
Wrote image 27
Wrote image 28
Wrote image 29
Wrote image 30
Wrote image 31
Wrote image 32
Wrote image 33
Wrote image 34
Wrote image 35
Wrote image 36
Wrote image 37
Wrote image 

# Dataset prepare

In [5]:
## remove empty images
## randomize filename

import json
import os
import random, string
from shutil import copyfile

def random_id():
  return ''.join(random.choices(string.ascii_letters + string.digits, k=16))

def init_final_dataset():
  import shutil
  
  shutil.rmtree('./annotated_datasets/final')
  os.mkdir('./annotated_datasets/final')
  os.mkdir('./annotated_datasets/final/images')
  os.mkdir('./annotated_datasets/final/annotations')

  initial_data = {"licenses": [{"name": "", "id": 0, "url": ""}], "info": {"contributor": "", "date_created": "", "description": "", "url": "", "version": "", "year": ""}, "categories": [{"id": 1, "name": "anymal", "supercategory": ""}, {"id": 2, "name": "spot", "supercategory": ""}],"images":[],"annotations":[]}
  
  with open('./annotated_datasets/final/annotations/annotations.json', 'w') as json_file:
    json.dump(initial_data, json_file)




def merge_to_final(dataset_path):
  output_dataset = "./annotated_datasets/final"
  output_images_path = output_dataset+"/images"


  #dataset_path = "annotated_datasets/rosbag-02"
  images_path = dataset_path+"/images/"
  with open(dataset_path+'/annotations/instances_default.json') as f:
    data = json.load(f)

  annotations = data["annotations"]
  images = data["images"]


  with open(output_dataset+'/annotations/annotations.json') as fo:
    output_data = json.load(fo)

  used_image_ids = []
  used_images = []

  annotation_counter = 0
  image_counter = 0

  for image in output_data["images"]:
    if image_counter < image["id"]:
      image_counter = image["id"]

  for annotation in output_data["annotations"]:
    if annotation_counter < annotation["id"]:
      annotation_counter = annotation["id"]

  image_counter += 1
  annotation_counter += 1

  used_annotations = []

  for annotation in annotations:
      used_image_ids.append(annotation['image_id'])
      _annotation = annotation
      _annotation["id"] = _annotation["id"] + annotation_counter
      _annotation["image_id"] = _annotation["image_id"] + image_counter
      used_annotations.append(_annotation)

  for image in images:
      if image["id"] in used_image_ids:
          _image = image
          id = random_id()+".png"
          copyfile(images_path+"/"+image["file_name"], output_images_path+"/"+id)
          _image["file_name"] = id
          _image["id"] = _image["id"] + image_counter
          used_images.append(_image)

  output_data["images"].extend(used_images)
  output_data["annotations"].extend(used_annotations)

  with open(output_dataset+'/annotations/annotations.json', 'w') as json_file:
    json.dump(output_data, json_file)




In [6]:
init_final_dataset()

datasets = [
    "rosbag-01",
    "rosbag-02",
    "chris_handy"
]

for dataset in datasets:
    merge_to_final(f"./annotated_datasets/{dataset}")

## Split Dataset

In [3]:
import json


annotation_folder = "annotated_datasets/final/annotations/annotations.json"


with open(annotation_folder) as f:
    data = json.load(f)

print(data)


train_data = {
    "licenses": data["licenses"],
    "categories": data["categories"],
    "images": [],
    "annotations":[]
}


{'licenses': [{'name': '', 'id': 0, 'url': ''}], 'info': {'contributor': '', 'date_created': '', 'description': '', 'url': '', 'version': '', 'year': ''}, 'categories': [{'id': 1, 'name': 'anymal', 'supercategory': ''}, {'id': 2, 'name': 'spot', 'supercategory': ''}], 'images': [{'id': 3, 'width': 1280, 'height': 720, 'file_name': 'GnOXOBfCncoqnjZE.png', 'license': 0, 'flickr_url': '', 'coco_url': '', 'date_captured': 0}, {'id': 4, 'width': 1280, 'height': 720, 'file_name': 'YEtI27aTCVMZPcBM.png', 'license': 0, 'flickr_url': '', 'coco_url': '', 'date_captured': 0}, {'id': 6, 'width': 1280, 'height': 720, 'file_name': 'Uhz14DxXKGFGCHiO.png', 'license': 0, 'flickr_url': '', 'coco_url': '', 'date_captured': 0}, {'id': 9, 'width': 1280, 'height': 720, 'file_name': '3wFmWJeSMKOohtxu.png', 'license': 0, 'flickr_url': '', 'coco_url': '', 'date_captured': 0}, {'id': 13, 'width': 1280, 'height': 720, 'file_name': 'Xn9Epo4WusUkbGPc.png', 'license': 0, 'flickr_url': '', 'coco_url': '', 'date_capt

In [7]:
from os import walk
from pathlib import Path


def convertDataset(img_path,ann_path,output_path):
  Path(output_path).mkdir(parents=True, exist_ok=True)

  !python3 ./convert2Yolo/example.py --datasets COCO --img_path {img_path} --label {ann_path} --convert_output_path {output_path} --img_type ".png" --manifest_path ./ --cls_list_file {coco_names}

  filenames = next(walk(output_path), (None, None, []))[2]  # [] if no file

  print(filenames)

  with open(output_path+"yolo_label.txt", "w") as file:
    for filename in filenames:
      file.write(filename)


dataset_path = "./annotated_datasets"
convertDataset("annotated_datasets/final/images","annotated_datasets/final/annotations/annotations.json","annotated_datasets/final/annotations/yolo/")

Traceback (most recent call last):
  File "/Users/dajo/Documents/GitHub/spot_anymal_dataset/./convert2Yolo/example.py", line 139, in <module>
    main(config)
  File "/Users/dajo/Documents/GitHub/spot_anymal_dataset/./convert2Yolo/example.py", line 57, in main
    flag, data, cls_hierarchy = coco.parse(
ValueError: not enough values to unpack (expected 3, got 2)
[]


## Data Augmentation Pipeline

In [3]:
!pip install -U albumentations

     |████████████████████████████████| 102 kB 2.6 MB/s eta 0:00:01
  Using cached typing_extensions-4.0.1-py3-none-any.whl (22 kB)
